# Question and Answer

A common use case for using large language models is to generate answers to questions based on a given piece of text. We will be focusing on the "extractive" Q&A use case, where we are providing the model with a piece of text we think is relevant to the question, and asking it to extract the answer out of it.

To start, we first need to instantiate an Aleph Alpha Client so that we can make calls to the Aleph Alpha API, which we use to get access to the Aleph Alpha family of models.

Make sure to create an account and an API token at [app.aleph-alpha.com](https://app.aleph-alpha.com), and set it as an environment variable or in a `.env` file in this repository.

In [ ]:
from os import getenv
from aleph_alpha_client import Client

client = Client(getenv("AA_TOKEN"))

Now, let's grab a piece of text we want to ask a question about. We can start with a random wikipedia article about ["Surface micromachining"](https://en.wikipedia.org/wiki/Surface_micromachining)

In [ ]:
text = """Surface micromachining

Surface micromachining builds microstructures by deposition and etching structural layers over a substrate.[1] This is different from Bulk micromachining, in which a silicon substrate wafer is selectively etched to produce structures.

Layers

Generally, polysilicon is used as one of the substrate layers while silicon dioxide is used as a sacrificial layer. The sacrificial layer is removed or etched out to create any necessary void in the thickness direction. Added layers tend to vary in size from 2-5 micrometres. The main advantage of this machining process is the ability to build electronic and mechanical components (functions) on the same substrate. Surface micro-machined components are smaller compared to their bulk micro-machined counterparts.

As the structures are built on top of the substrate and not inside it, the substrate's properties are not as important as in bulk micro-machining. Expensive silicon wafers can be replaced by cheaper substrates, such as glass or plastic. The size of the substrates may be larger than a silicon wafer, and surface micro-machining is used to produce thin-film transistors on large area glass substrates for flat panel displays. This technology can also be used for the manufacture of thin film solar cells, which can be deposited on glass, polyethylene terepthalate substrates or other non-rigid materials.

Fabrication process

Micro-machining starts with a silicon wafer or other substrate upon which new layers are grown. These layers are selectively etched by photo-lithography; either a wet etch involving an acid, or a dry etch involving an ionized gas (or plasma). Dry etching can combine chemical etching with physical etching or ion bombardment. Surface micro-machining involves as many layers as are needed with a different mask (producing a different pattern) on each layer. Modern integrated circuit fabrication uses this technique and can use as many as 100 layers. Micro-machining is a younger technology and usually uses no more than 5 or 6 layers. Surface micro-machining uses developed technology (although sometimes not enough for demanding applications) which is easily repeatable for volume production.

Sacrificial layers

A sacrificial layer is used to build complicated components, such as movable parts. For example, a suspended cantilever can be built by depositing and structuring a sacrificial layer, which is then selectively removed at the locations where the future beams must be attached to the substrate (i.e. the anchor points). A structural layer is then deposited on top of the polymer and structured to define the beams. Finally, the sacrificial layer is removed to release the beams, using a selective etch process that does not damage the structural layer.

Many combinations of structural and sacrificial layers are possible. The combination chosen depends on the process. For example, it is important for the structural layer not to be damaged by the process used to remove the sacrificial layer.

Examples

Surface Micro-machining can be seen in action in the following MEMS (Microelectromechanical) products:

- Surface Micro-machined Accelerometers[2]
- 3D Flexible Multichannel Neural Probe Array[3]
- Nanoelectromechanical relays"""

Now, we can start with an example Q&A workflow we've provided. The main things of interest are that you can provide a `QaInput`, which consists of a `question` you want to ask, and a `text` to base that answer upon.

The output will be a `QaOutput`, which will include an `answer` (if it can find one in the text), as well as a `debug_log`, which we will go into later.

In [ ]:
from intelligence_layer.single_chunk_qa import SingleChunkQa, SingleChunkQaInput
from IPython.display import Pretty

question = "What are some benefits of surface micro-machining?"

qa = SingleChunkQa(client, "debug", model="luminous-supreme-control")
input = SingleChunkQaInput(chunk=text, question=question)
output = qa.run(input)

Pretty(output.answer)

In [ ]:
output.highlights

In [ ]:
output.debug_log

QA becomes slightly more complicated with multiple chunks.
The workflow in this case has two steps.
First the model generates an answer based on each chunk and the given question.
Then all the intermediate answers are combined into a singular answer.
This is the final given answer.

Below you can see an example of this where two chunks provide different parts of the final answer.



In [ ]:
from intelligence_layer.multiple_chunk_qa import MultipleChunkQa, MultipleChunkQaInput

chunks = ["As the sun dipped below the horizon, the lively sounds of laughter and clinking beer steins filled the air at the 213th festival edition. The festival grounds were adorned with colorful banners and adorned tables, where people from all over the world gathered to celebrate. The aroma of sizzling sausages and freshly baked pretzels wafted from the food stalls, tempting hungry visitors.",
          "Musicians played traditional Bavarian tunes, and couples twirled on the makeshift dance floor, lost in the joy of the moment. Under the starry sky, Oktoberfest was a testament to the enduring spirit of camaraderie and merriment."]

question = "What festival is the text about?"

qa = MultipleChunkQa(client, "debug", model="luminous-supreme-control")
input = MultipleChunkQaInput(chunks=chunks, question=question)
output = qa.run(input)

Now if we have a look at the intermediate answers in the debug-log:

In [ ]:

from intelligence_layer.task import InfoEnabledLog 

result_objects = [log_entry for log_entry in output.debug_log.root if log_entry.message == "Intermediate Answers"]

InfoEnabledLog(root=result_objects)

After another completion request where we asked the model to combine the intermediate answers we get the final answer.

In [ ]:
Pretty(output.answer)